In [5]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sci
import scipy.integrate as integrate
import scipy.stats as stats

### Parameters and initial coditions

In [7]:
kp = 1.1
km = 0.1
k2 = 0.8
S0=100
E0 =100
C0=1
P0=1
t=0

### Gillespie algorithm

### Euler method

In [8]:
t_end= 10000
step_size = 0.1

In [ ]:
def R_Euler(kp, km , k2 ,S0, E0 , C0 , P0):
    
    #initial setting
    t_array = np.zeros( int(t_end/step_size) +1 )
    S_array = np.zeros( int(t_end/step_size) +1 )
    E_array = np.zeros( int(t_end/step_size) +1 )
    C_array = np.zeros( int(t_end/step_size) +1 )
    P_array = np.zeros( int(t_end/step_size) +1 )    
    
    j=0
    t_array[j] = 0
    S_array[j] = S0
    E_array[j] = E0
    C_array[j] = C0
    P_array[j] = P0
    #start the process
    while t <= t_end :
        j= j+1
        Wp  , Wm , W2 = np.random.normal(0,step_size**2,3)
        dS = (-kp*S*E + km *C)*step_size + ( -np.sqrt(kp*S*E)*Wp + np.sqrt(km*C)*Wm)
        dE = (-kp*S*E + km *C + k2*C)*step_size + ( -np.sqrt(kp*S*E)*Wp + np.sqrt(km*C)*Wm + np.sqrt(k2*C)*W2)
        dC = -dE
        dP = km *C*step_size+ np.sqrt(k2*C)*W2
        
        S = S + dS
        E = E + dE
        C = C + dC
        P = P + dP
        t = t + step_size
        t_array[j] = t
        S_array[j] = S
        E_array[j] = E
        C_array[j] = C
        P_array[j] = P
        
    return t_array , S_array , E_array , C_array  , P_array 

### Tau-leaping method

### Two time scale stochastic simulation